In [ ]:
! pip install /content/llm_comparator-0.1-py3-none-any.whl

In [ ]:
import vertexai
from google.colab import auth

from llm_comparator import comparison
from llm_comparator import model_helper
from llm_comparator import llm_judge_runner
from llm_comparator import rationale_bullet_generator
from llm_comparator import rationale_cluster_generator

In [ ]:
#@title Setup for using Vertex AI.
auth.authenticate_user()

PROJECT_ID = 'pair-experimental'  #@param {type: "string"}
REGION = 'us-central1'  #@param {type: "string"}

! gcloud config set project {PROJECT_ID}

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
llm_judge_inputs = [
    {'prompt': 'how are you?', 'response_a': 'good', 'response_b': 'bad'},
    {'prompt': 'hello?', 'response_a': 'hello', 'response_b': 'hi'},
    {'prompt': 'what is the capital of korea?', 'response_a': 'Seoul', 'response_b': 'Vancouver'}
]

In [ ]:
#@title Initialize models used in the LLM Comparator evaluation.
generator = model_helper.VertexGenerationModelHelper()
embedder = model_helper.VertexEmbeddingModelHelper()
judge = llm_judge_runner.LLMJudgeRunner(generator)
bulletizer = rationale_bullet_generator.RationaleBulletGenerator(generator)
clusterer = rationale_cluster_generator.RationaleClusterGenerator(
    generator, embedder
)

In [ ]:
#@title Run the LLM Comparator evauation.
comparison_result = comparison.run(
    llm_judge_inputs,
    judge,
    bulletizer,
    clusterer,
)

In [ ]:
file_path = 'json_for_llm_comparator.json' # @param {type: "string"}
comparison.write(comparison_result, file_path)

In [ ]:
! git clone https://github.com/PAIR-code/llm-comparator

In [ ]:
#@title For displaying LLM Comparator.
import IPython

# TODO: Move to the pip package.
def show_llm_comparator(json_path, height=800, port=4321):
  IPython.get_ipython().system_raw(f'python3 -m http.server {port} &')
  IPython.display.display(IPython.display.Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%s)
    results_path = fm.src + '%s'
    fm.src += 'llm-comparator/docs/'
    fm.src += '?results_path=' + results_path
    fm.width = '100%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, json_path, height) ))

In [ ]:
show_llm_comparator(file_path, port=7676)